## Assignment 2: Adversarial Training

This assignment requires you to create adversarial examples. You will do adversarial training, i.e., train the model with sets of adversarial examples you generated and evaluate the performances of the model on test sets.

### What is Adversarial Training?
Adversarial training is a machine learning technique to improve models' robustness by training them on adversarial examples. Adversarial examples are input data that has been intentionally modified to cause the model to misclassify or produce an incorrect output.

When a model is trained using adversarial examples, it becomes more resilient to adversarial attacks and is able to better identify and classify input data that may have been modified or corrupted. This may lead to improved performance of the model in real-world scenarios where the input data may not always be perfect.

However, adversarial training can also have some negative impacts on ML models. For example, it can lead to overfitting, where the model becomes too specialized to the particular adversarial examples used in training and is unable to generalize well to new examples. Additionally, adversarial training can increase the computational requirements of training the model due to the need for generating adversarial examples.

Overall, while adversarial training can improve the robustness of ML models, it is important to carefully consider its potential benefits and drawbacks and to evaluate the trade-offs in terms of model performance and computational requirements. The following are the steps involved in adversarial training:

1. Generate adversarial examples: In the first step, we generate adversarial examples by perturbing the original data points in such a way that the modifications are small and not noticeable to humans but are enough to cause misclassification by the neural network.

2. Train on adversarial examples: In the second step, we train the neural network on adversarial examples in addition to the original training data. This helps to improve the network's ability to recognize and classify adversarial examples correctly.

3. Evaluate performance: In the third step, we evaluate the performance of the network on both the original and adversarial test data. This helps to determine if the adversarial training has improved the network's robustness against adversarial attacks.

Overall, adversarial training is a powerful technique that can help improve the security and reliability of deep neural networks.

In this Homework, you will run different adversarial training algorithms on the ResNet18 model with the adversarial examples, and evaluating the model performances on test data. The goal is to get experience in generating adversarial examples and train the model with these examples, i.e., adversarial training.

We have provided the model architecture ($\texttt{model.py}$) and some pre-defined functions ($\texttt{utils.py}$) so you can import and use them directly in the notebook.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import numpy as np
from model import ResNet18
from utils import trades_loss, mixup_data, mixup_criterion, make_dataloader, eval_test

device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Q1 (20 points)
Use the following parameters to define the LinfPGDAttack():

- Epsilon: 8/255
- PGD Steps: 10
- PGD Step Size: 0.003

In [ ]:
import torch
import torch.nn as nn

class LinfPGDAttack(nn.Module):
    def __init__(self, model, epsilon, steps=10, step_size=0.003):
        super(LinfPGDAttack, self).__init__()
        self.model = model
        self.epsilon = epsilon
        self.steps = steps
        self.step_size = step_size

    def perturb(self, x_natural, y, device='cuda'):
        """
        Computes the gradient of the cross-entropy loss with respect to the input
        image `x_adv` and updates the image based on the gradient direction. The
        perturbation is clipped to ensure it stays within a specified epsilon range
        and is finally clamped to ensure pixel values are valid.

        The resulting perturbed image is returned.
        """
        # *********** Your code starts here ***********
        attacked = x_natural.clone().detach()
        attacked.to(device)
        attacked.requires_grad = True
        criterion = torch.nn.CrossEntropyLoss()
        for _ in range(self.steps):
            output = self.model(attacked)
            loss = criterion(output, y)
            loss.backward()
            gradient_sign = attacked.grad.sign()
            attacked.grad.zero_()
            attacked = attacked + self.step_size * gradient_sign
            attacked = torch.max(torch.min(attacked, x_natural + self.epsilon), x_natural - self.epsilon)
            attacked = attacked.clamp(0, 255)
            attacked = attacked.clone().detach()
            attacked.to(device)
            attacked.requires_grad = True
        # *********** Your code ends here *************
        return attacked

    def forward(self, x_natural, y, device='cuda'):
        x_adv = self.perturb(x_natural, y, device)
        return x_adv


There are many implementations of adversarial training; in this assignment, we ask you to evalaute which training algorithm can make the model more robust to LinfPGDAttack().

In [ ]:
def train_ep(model, train_loader, mode, pgd_attack, optimizer, criterion, epoch, batch_size):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        if mode == 'natural':
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)

        elif mode == 'adv_train': # [Ref] https://arxiv.org/abs/1706.06083
            model.eval()
            adv_x = pgd_attack(inputs, targets)
            model.train()

            optimizer.zero_grad()
            outputs = model(adv_x)
            loss = criterion(outputs, targets)

        elif mode == 'adv_train_trades': # [Ref] https://arxiv.org/abs/1901.08573
            optimizer.zero_grad()
            loss = trades_loss(model=model, x_natural=inputs, y=targets, optimizer=optimizer)

        elif mode == 'adv_train_mixup': # [Ref] https://arxiv.org/abs/1710.09412
            model.eval()
            benign_inputs, benign_targets_a, benign_targets_b, benign_lam = mixup_data(inputs, targets)
            adv_x = pgd_attack(inputs, targets)
            adv_inputs, adv_targets_a, adv_targets_b, adv_lam = mixup_data(adv_x, targets)
            model.train()
            optimizer.zero_grad()
            benign_outputs = model(benign_inputs)
            adv_outputs = model(adv_inputs)
            loss_1 = mixup_criterion(criterion, benign_outputs, benign_targets_a, benign_targets_b, benign_lam)
            loss_2 = mixup_criterion(criterion, adv_outputs, adv_targets_a, adv_targets_b, adv_lam)

            loss = (loss_1 + loss_2) / 2

        else:
            print("No training mode specified.")
            raise ValueError()

        loss.backward()
        optimizer.step()

        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{:05d}/{} ({:.0f}%)]\t Loss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(inputs), len(train_loader) * batch_size,
                       100. * (batch_idx + 1) / len(train_loader), loss.item()))

In [ ]:
def train(model, train_loader, val_loader, pgd_attack,
          mode='natural', epochs=25, batch_size=256, learning_rate=0.1, momentum=0.9, weight_decay=2e-4,
          checkpoint_path='model1.pt'):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

    best_acc = 0
    for epoch in range(epochs):
        # training
        train_ep(model, train_loader, mode, pgd_attack, optimizer, criterion, epoch, batch_size)
        # evaluate clean accuracy
        test_loss, test_acc = eval_test(model, val_loader, device)
        # remember best acc@1 and save checkpoint
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        # save checkpoint if is a new best
        if is_best:
            torch.save(model.state_dict(), checkpoint_path)
        print('================================================================')

### Q2 (40 points)
Use the four training modes ("natural", "adv_train", "adv_train_trades", and "adv_train_mixup") to obtain four models, and save them as $\texttt{model1.pt}$, $\texttt{model2.pt}$, $\texttt{model3.pt}$, and $\texttt{model4.pt}$, respectively.

When calculating your losses, you may encounter Nan. In this case, you may consider adjusting the `learning rate` to solve the problem.

In [ ]:
# define parameters
batch_size = 256
data_path = "../data" # directory of the data
epsilon = 8/255
steps = 10
epochs = 25
# create data loader
train_loader, val_loader = make_dataloader(data_path, batch_size)

100%|██████████| 170498071/170498071 [00:02<00:00, 72516926.67it/s]


Extracting ../data/cifar-10-python.tar.gz to ../data
Files already downloaded and verified


In [ ]:
training_mode = "natural"
# Define Model and Launch Training (Define Adversary If You Need It)
model1 = ResNet18()
model1.to(device)
train(model1, train_loader, val_loader, LinfPGDAttack(model1, epsilon), training_mode, epochs, checkpoint_path = "model1.pt")
# Write your code here

In [ ]:
training_mode = "adv_train"
# Define Model and Launch Training (Define Adversary If You Need It)
model2 = ResNet18().to(device)
train(model2, train_loader, val_loader, LinfPGDAttack(model2, epsilon), training_mode, epochs, checkpoint_path = "model2.pt")
# Write your code here

In [ ]:
training_mode = "adv_train_trades"
# Define Model and Launch Training (Define Adversary If You Need It)
model3 = ResNet18()
model3.to(device)
model3.load_state_dict(torch.load(os.getcwd() + "/model1.pt"))
train(model3, train_loader, val_loader, LinfPGDAttack(model3, epsilon), training_mode, epochs, checkpoint_path = "model3.pt")
# Write your code here

In [ ]:
training_mode = "adv_train_mixup"
# Define Model and Launch Training (Define Adversary If You Need It)
model4 = ResNet18()
model4.to(device)
model4.load_state_dict(torch.load(os.getcwd() + "/model1.pt"))
train(model4, train_loader, val_loader, LinfPGDAttack(model4, epsilon), training_mode, epochs, checkpoint_path = "model4.pt")
# Write your code here

In the following file, I am loading models because they were not trained from this file, so I can't use their instances

### Q3 (20 points)
Use eval_robust() to evaluate each model's robustness against LinfPGDAttack().

In [ ]:
from utils import eval_robust
import os
# *********** Your code starts here ***********
checkpoint = torch.load(os.getcwd() + "/model1.pt")
model1 = ResNet18()
model1.load_state_dict(checkpoint)
model1.to(device)
model1.load_state_dict(torch.load(os.getcwd() + "/model1.pt"))
adversary = LinfPGDAttack(model1, epsilon)
robust_loss, robust_acc = eval_robust(model1, val_loader, adversary)
# *********** Your code ends here *************

In [ ]:
# *********** Your code starts here ***********
checkpoint = torch.load(os.getcwd() + "/model2.pt")
model = ResNet18()
model.load_state_dict(checkpoint)
model.to(device)
adversary = LinfPGDAttack(model, epsilon)
robust_loss, robust_acc = eval_robust(model, val_loader, adversary)
# *********** Your code ends here *************

In [ ]:
# *********** Your code starts here ***********
checkpoint = torch.load(os.getcwd() + "/model3.pt")
model = ResNet18()
model.load_state_dict(checkpoint)
model.to(device)
adversary = LinfPGDAttack(model, epsilon)
robust_loss, robust_acc = eval_robust(model, val_loader, adversary)
# *********** Your code ends here *************

In [ ]:
# *********** Your code starts here ***********
checkpoint = torch.load(os.getcwd() + "/model4.pt")
model = ResNet18()
model.load_state_dict(checkpoint)
model.to(device)
adversary = LinfPGDAttack(model, epsilon)
robust_loss, robust_acc = eval_robust(model, val_loader, adversary)
# *********** Your code ends here *************

### Q4 (10 points)
Visualize 10 adversarial examples from (with ground truth labels and model's predictions) from the model having best robust accuarcy.

In [ ]:
# *********** Your code starts here ***********
import matplotlib.pyplot as plt
assuming model4 has the bes accuracy. I ran out of training computers, and was not able to continue training all the models
checkpoint = torch.load(os.getcwd() + "/model4.pt")
best_model = ResNet18()
best_model.load_state_dict(checkpoint)
best_model.to(device)
adversary = LinfPGDAttack(best_model, epsilon)
for index, (input, label)in enumerate(val_loader):
  count = 0
  for image, label in zip(input, label):
    image = adversary(image, label)
    plt.imshow(image.T)
    count += 1
    if count >= 9:
      break
  break
# *********** Your code ends here *************

### Q5 (10 points)
Which adversarial training algorithm achieves the best robust accuracy in only 25 training epochs? Why do you think that adversarial training algorithm outperforms others? Do you encounter any difficulties when you implement this assignment?

**NOTE:** I did not train for 25 epochs. For every model I trained for around 10 epochs. Also, I reached my GPU usage at Google Colab, so I couldn't run the robustness tests. So I am going to do analogies based on the algorithms themselves, and what the content learned in class.

 **The best algorithm:** Should be between TRADES and adv_train_mixup. This is because, on the top of adversarial training, adv_train_mix up includes the regularization methods which can improve the accuracy of the model, while TRADES maintains the balance between achieveing high accuracy on clean data, and maintaining the robustness of the model on adversarial attacks.


**Reasons**

At the top of making adversarial attacks, the algorithms includes data augmentation techniques which reduces overfitting, and improves robustness.

## Submission Instructions

Please submit this  to Canvas, as well as results, as per instructions.
Please compress `First_Middle_Last_HW1/` into one zip file with the name `First_Middle_Last_HW1.zip` before uploading it to Canvas. The directory contains your notebook and four model checkpoints. As listed below:

- $\texttt{Assignment_2.ipynb}$: your code
- $\texttt{model1.pt}$: your model checkpoint with *natural training*
- $\texttt{model2.pt}$: your model checkpoint with *adv_train* training
- $\texttt{model3.pt}$: your model checkpoint with *adv_train_trades* training
- $\texttt{model4.pt}$: your model checkpoint with *adv_train_mixup* training

## Academic Integrity

This homework assignment must be done individually. Sharing code or model specifications is strictly prohibited. Homework discussions are allowed only on Piazza, according to the policy outlined on the course web page: [https://canvas.dartmouth.edu/courses/63219](https://canvas.dartmouth.edu/courses/63219). You are not allowed to search online for auxiliary software, reference models, architecture specifications, or additional data to solve the homework assignment. Your submission must be entirely your own work. That is, the code and the answers that you submit must be created, typed, and documented by you alone, based exclusively on the materials discussed in class, and released with the homework assignment. You can obviously consult the class slides posted in Canvas, your lecture notes, and the textbook. Important: the models you will submit for this homework assignment must be
trained exclusively on the specified data provided with this assignment. You can, of course, play with other datasets in your spare time. These rules will be strictly enforced, and any violation will be treated seriously.